In [210]:
import torch

def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index],
                      db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

def query(db, threshold=5):
    return (db.sum() > threshold).float()

def sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
            
    return sensitivity

In [211]:
db, pdbs = create_db_and_parallels(100)
db

tensor([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0], dtype=torch.uint8)

In [212]:
true_result = torch.mean(db.float())
true_result

tensor(0.6200)

In [213]:
def query(db):
    public_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    augumented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    private_result = torch.mean(augumented_database.float()) * 2 - 0.5
    
    return private_result, public_result

In [214]:
db, pdbs = create_db_and_parallels(10)
private_result, public_result = query(db)

print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.5000)
Without noise:tensor(0.5000)


In [215]:
db, pdbs = create_db_and_parallels(100)
private_result, public_result = query(db)

print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.6400)
Without noise:tensor(0.5600)


In [216]:
db, pdbs = create_db_and_parallels(1000)
private_result, public_result = query(db)

print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.5160)
Without noise:tensor(0.4850)


In [217]:
db, pdbs = create_db_and_parallels(10000)
private_result, public_result = query(db)

print("With noise:" + str(private_result))
print("Without noise:" + str(public_result))

With noise:tensor(0.4910)
Without noise:tensor(0.4972)
